In [1]:
import os
import sys
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\qhj\ZERO")

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine

from tools import *


In [2]:
file_path = r"C:\Users\zero\Desktop\TEST.xlsx"

df = pd.read_excel(file_path)
data = df.copy()
data['grandfather_id'] = 0
data['grandfather'] = None
data['level'] = 1
data['root'] = None
data['root_name'] = None
data['二级佣金'] = 0
data['三级佣金'] = 0
data['无限极佣金'] = 0


commission_dic = {
    "VIP":20,
    "VVIP":50
}

data['root'][data['parent_id'] == 0] = data[data['parent_id'] == 0]['user_id']
data['root_name'][data['parent_id'] == 0] = data[data['parent_id'] == 0]['user_name']
data = data.set_index('user_id',drop=False)

In [3]:
data[:3]

,user_id,user_name,parent_name,parent_id,state,grandfather_id,grandfather,level,root,root_name,二级佣金,三级佣金,无限极佣金
user_id,,,,,,,,,,,,,
1,1,A1,root,0,合伙人,0,None,1,1,A1,0,0,0
2,2,A2,root,0,VVIP,0,None,1,2,A2,0,0,0
3,3,B1,A1,1,VVIP,0,None,1,None,None,0,0,0


In [4]:
data.loc[[1,2],['user_name','parent_name']]

,user_name,parent_name
user_id,,
1,A1,root
2,A2,root


In [5]:
for index,row in data.iterrows():
    parent_id = row['parent_id']
    state = row['state']
    commission = commission_dic.get(state,0)
    

    # 二级
    if parent_id:

        data.loc[index,'level'] += data.loc[parent_id,'level']
        data.loc[index,'root'] = data.loc[parent_id,'root']
        data.loc[index,'root_name'] = data.loc[parent_id,'root_name']
        
        data.loc[parent_id,'二级佣金'] +=  commission
        data.loc[parent_id,'无限极佣金'] +=  commission
        
        parent_id = data.loc[parent_id,'parent_id']
    
    # 三级
    if parent_id:
        data.loc[parent_id,'三级佣金'] += commission
        data.loc[parent_id,'无限极佣金'] +=  commission
        
        data.loc[index,'grandfather_id'] =  parent_id
        data.loc[index,'grandfather'] =  data.loc[parent_id,'user_name']
        
        parent_id = data.loc[parent_id,'parent_id']

    while parent_id:
        data.loc[parent_id,'无限极佣金'] +=  commission
        parent_id = data.loc[parent_id,'parent_id']

        


In [6]:
s = set(data['grandfather_id'].unique())
s.remove(0)
s

{1, 2, 4, 5, 8, 9}

In [50]:
data.to_excel(r"C:\Users\zero\Desktop\T\处理后的数据.xlsx",index=False)

In [47]:
temp = data[['state','user_id', 'user_name', 'parent_id', 'parent_name', 'grandfather', 'grandfather_id']]. \
dropna().rename(columns={"user_name":"三级用户名",
                         'state':"三级用户等级",
                         'parent_name':"二级用户名"})
temp['可抽取佣金'] = temp['三级用户等级'].apply(lambda x:commission_dic.get(x,0))

In [49]:
save_dir = r"C:\Users\zero\Desktop\T\需要发送的三级名单"
for (grandfather,grandfather_id),t in temp.groupby(['grandfather','grandfather_id']):
    t=  t[['二级用户名','三级用户名','三级用户等级','可抽取佣金']].sort_values('二级用户名')
    save_path = os.path.join(save_dir, '%s_%s.xlsx' %(grandfather_id,grandfather))
    t.to_excel(save_path, index=False)
    

In [7]:
data[['user_id', 'user_name', 'pareant_id', 'parent_name', 'grandfather', 'grandfather_id']]

,user_id,user_name,parent_name,parent_id,state,grandfather_id,grandfather,level,root,root_name,二级佣金,三级佣金,无限极佣金
user_id,,,,,,,,,,,,,
1,1,A1,root,0,合伙人,0,None,1,1,A1,170,190,480
2,2,A2,root,0,VVIP,0,None,1,2,A2,150,220,490
3,3,B1,A1,1,VVIP,0,None,2,1,A1,20,0,20
4,4,B2,A1,1,VVIP,0,None,2,1,A1,70,70,140
5,5,B3,A1,1,VVIP,0,None,2,1,A1,100,50,150
6,6,B4,A1,1,VIP,0,None,2,1,A1,0,0,0
7,7,B5,A2,2,VVIP,0,None,2,2,A2,20,0,20
8,8,B6,A2,2,VVIP,0,None,2,2,A2,100,70,170
9,9,B7,A2,2,VVIP,0,None,2,2,A2,100,50,150


In [8]:
data

,user_id,user_name,parent_name,parent_id,state,grandfather_id,grandfather,level,root,root_name,二级佣金,三级佣金,无限极佣金
user_id,,,,,,,,,,,,,
1,1,A1,root,0,合伙人,0,None,1,1,A1,170,190,480
2,2,A2,root,0,VVIP,0,None,1,2,A2,150,220,490
3,3,B1,A1,1,VVIP,0,None,2,1,A1,20,0,20
4,4,B2,A1,1,VVIP,0,None,2,1,A1,70,70,140
5,5,B3,A1,1,VVIP,0,None,2,1,A1,100,50,150
6,6,B4,A1,1,VIP,0,None,2,1,A1,0,0,0
7,7,B5,A2,2,VVIP,0,None,2,2,A2,20,0,20
8,8,B6,A2,2,VVIP,0,None,2,2,A2,100,70,170
9,9,B7,A2,2,VVIP,0,None,2,2,A2,100,50,150
